In [ ]:
%pip install -e ..
%restart_python

In [ ]:
import json
import requests
import sys
from loguru import logger
from pyspark.sql import SparkSession

from pyspark.dbutils import DBUtils
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:


workspace_url = dbutils.widgets.get("workspace_url")
endpoint_name = dbutils.widgets.get("endpoint_name")
# token = dbutils.secrets.get("mlops", "databricks-pat")

payload_path = "../tests/battery_test.json"

with open(payload_path) as f:
    payload = json.load(f)

# url = f"https://{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

# headers = {
#     "Authorization": f"Bearer {token}",
#     "Content-Type": "application/json"
# }
headers = {
    "Authorization": f"Bearer {dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}",
    "Content-Type": "application/json",
}

resp = requests.post(endpoint_name, headers=headers, json=payload, timeout=300)

logger.info("Status: {}", resp.status_code)
logger.info("Response: {}", resp.text)

if resp.status_code != 200:
    raise RuntimeError("❌ Serving smoke test failed")

logger.info("✅ Serving smoke test passed")


In [ ]:
# import json
# import requests
# import time
# from databricks.sdk import WorkspaceClient

# # Initialize Databricks Client
# w = WorkspaceClient()

# workspace_url = dbutils.widgets.get("workspace_url")
# endpoint_name = dbutils.widgets.get("endpoint_name")
# payload_path = "../tests/battery_test.json"

# with open(payload_path) as f:
#     payload = json.load(f)

# # --- NEW: Status Check Logic ---
# logger.info(f"Checking status for endpoint: {endpoint_name}")
# max_retries = 10
# for i in range(max_retries):
#     status = w.serving_endpoints.get(endpoint_name)
#     state = status.state.ready
    
#     if state == "READY":
#         logger.info("✅ Endpoint is READY. Proceeding to smoke test.")
#         break
#     else:
#         logger.warning(f"Status is {state}. Waiting 30s (Attempt {i+1}/{max_retries})...")
#         time.sleep(30)
# else:
#     raise RuntimeError(f"❌ Endpoint failed to reach READY state after {max_retries} attempts.")
# # -------------------------------

# # Construct the correct URL (Fixing the previous issue)
# url = f"https://{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

# headers = {
#     "Authorization": f"Bearer {dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}",
#     "Content-Type": "application/json",
# }

# # Increased timeout to 300 to account for cold-start wake-up time
# logger.info("Sending request to endpoint...")
# resp = requests.post(url, headers=headers, json=payload, timeout=300)

# logger.info("Status: {}", resp.status_code)
# logger.info("Response: {}", resp.text)

# if resp.status_code != 200:
#     raise RuntimeError(f"❌ Serving smoke test failed with status {resp.status_code}")

# logger.info("✅ Serving smoke test passed")